# Keyword Search

In [34]:
import time
from typing import List
import uuid

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
# Load in the data and add uuids
data = pd.read_json("data/preprocessed_data.jsonl", orient='records', lines=True)

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   uuid               209527 non-null  object        
 1   link               209527 non-null  object        
 2   headline           209527 non-null  object        
 3   category           209527 non-null  object        
 4   short_description  209527 non-null  object        
 5   authors            209527 non-null  object        
 6   date               209527 non-null  datetime64[ns]
 7   clean_headline     209527 non-null  object        
 8   combined_text      209527 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 14.4+ MB


In order to help speed up and simplify some of the code rather then relying on pandas for looking up articles I will use dictionaries instead.

In [19]:
# Initialize lookups including uuid -> title and row index -> uuid
UUID_2_TITLE = dict(zip(data['uuid'], data['headline']))
IND_2_UUID = dict(zip(list(data.index), data['uuid']))

# Create generic docs list to be used for setting up searchs.
DOCS = list(data['combined_text'])


## Naive Approach

In the naive approach we will simply check each document to see if it contains the query term/s. The main reason for calling this the "naive" approach is because it will utilize a for loop to check each document for the term/s. In a small sample such as this using a simple for loop works relatively quickly while very clearly showing how keyword matching works however with larger datasets with not only more records but also longer texts this approach is simply unrealistic due to its limitation in regards to processing time.

In [20]:
class KeywordSearcher:
    def __init__(self, docs:List[str]):
        self.docs : List[str] = docs

    def search(self, query:str)->List[str]:
        matches = [d for d in self.docs if query.lower() in d]
        return matches

In [21]:
%%time

searcher = KeywordSearcher(docs=DOCS)
results = searcher.search(query="airlines")
print(len(results))


396
CPU times: user 86.4 ms, sys: 207 µs, total: 86.6 ms
Wall time: 96 ms


In [22]:
for r in results[:3]:
    print(r + "\n")

american airlines flyer charged banned for life after punching flight attendant on video he was subdued by passengers and crew when he fled to the back of the aircraft after the confrontation according to the us attorney's office in los angeles

alaska airlines cancels dozens of flights as pilots picket more than 100 alaska airlines flights were canceled by the airline including 66 in seattle 20 in portland oregon 10 in los angeles and seven in san francisco

russia's flagship airline aeroflot halts all international flights except to belarus russia's aviation agency recommended all russian airlines with foreignleased planes halt both passenger and cargo flights abroad



As can be seen this performs well enough in regards to surfacing documents that contain the search term (i.e. "airlines") however as noted this approach would not be feasible on larger datasets and also comes into issues when dealing with longer search terms. In other words once the query gets longer (e.g. "airlines in the united states") this will introduce to many complexities to really be handled via a simple term lookup because then you may need to start testing strategies that split up the query into individual terms and search against each word at a time which again leads to poor performance in terms of lookup speeds. Therefore in an attempt to try and solve for this we will perform keyword search against vectorized documents.

## Vectorized Search

First we will start by creating an index class and a search class. While it isn't required to keep these seperate having them in seperate classes can help to customize things as needed. 

In [36]:
class KeywordIndex:
    
    def __init__(self, transformer_type:str):
        if transformer_type in ['count', 'tfidf']:
            if transformer_type == 'count':
                self.transformer = CountVectorizer()
            elif transformer_type == 'tfidf':
                self.transformer = TfidfVectorizer()
        else:
            raise ValueError(f"Expected count or tfidf but got {transformer_type}")
        # Sparse matrix representations of the docs to be queried
        self.index = None
        # DataFrame containing keys and other info for the docs
        self.lookup : pd.DataFrame = None
        # Column that will be returned when searching the index. Defining this 
        # allows us to return something other then the docs such as the title or an id.
        self.lookup_key : str = None

    def build(self, docs:List[str], lookup:pd.DataFrame, key_col:str):
        """Build the KeywordIndex by transforming the docs given the specified transformer. 
        """
        st = time.time()
        self.index = self.transformer.fit_transform(docs)
        self.lookup = lookup
        self.lookup_key = key_col 
        print(f"Time to build index : {round(time.time()-st,4)} seconds")
        return
    
    def search(self, query:str, n:int=5):
        """Searches the KeywordIndex using cosine similarity and returns tops N
        similar results. 
        """
        transformed_query = self.transformer.transform([query])
        similarity_matrix = cosine_similarity(transformed_query, self.index)
        self.lookup['tfidf_cos_sim'] = similarity_matrix[0]
        self.lookup.sort_values(by='tfidf_cos_sim',ascending=False,inplace=True)
        
        return list(self.lookup[self.lookup_key].head(n)) 
    
    

### Count Vectorization

In [37]:
# Build the index using a count transformer
index = KeywordIndex(transformer_type='count')
index.build(docs=DOCS, lookup=data, key_col='uuid')



Time to build index : 5.4211 seconds


In [38]:
%%time 

# search the index
res = index.search(query='airlines')

for cnt, u in enumerate(res):
    print(f"{cnt}: {UUID_2_TITLE[u]}")

0: Right Word, Wrong Basket: Keeping the Focus on Deplorable Language and Views
1: My Encounter With Merle
2: U.S. Tightens Screening Of Airport Workers After Gun Arrest
3: 3 Recipes For Anyone Who Loves Vegetables
4: The Way Out Of Trumpland: Electoral College And Clinton Watch Vote Recounts
CPU times: user 428 ms, sys: 26.9 ms, total: 455 ms
Wall time: 483 ms


Now because we are using cosine_similarity and have vectorized our scoreing mechanism it allows us to pass in longer queries and not worry as much about compute time as compared the naive approach. 

In [39]:
%%time 

# search the index
res = index.search(query='united states government')

for cnt, u in enumerate(res):
    print(f"{cnt}: {UUID_2_TITLE[u]}")

0: Women in Business Q&A: Shelley Zalis, Founder of The Girls' Lounge
1: Hikers’ Families Say Deaths Motivated By ‘Compassion’
2: Congress Members -- Shameless
3: Climate Change Poses A Big Risk To Your Retirement Savings
4: Sarah Silverman To Trump: 'Show Us Your F***king Taxes, You Emotional Child'
CPU times: user 427 ms, sys: 26.6 ms, total: 453 ms
Wall time: 499 ms


As we can see from the results above we are getting decent results however a downfall of keyword search is that it can fail to capture actual meaning or what a document is about. In other words although a document may contain the term it may not be about the term. For example given the query Washington D.C. a keyword based search may surface a document about the US government just because it makes mention of the US government being based in Washington D.C. without any other information about Washington D.C.

### tfidf Vectorization

A variation on keyword search that attempts to 

In [40]:
# Build the index using a count transformer
index = KeywordIndex(transformer_type='tfidf')
index.build(docs=DOCS, lookup=data, key_col='uuid')


Time to build index : 5.596 seconds


In [41]:
%%time 

# search the index
res = index.search(query='airlines')

for cnt, u in enumerate(res):
    print(f"{cnt}: {UUID_2_TITLE[u]}")

0: A Harvard Professor On The Profound Life Lessons Of 'Star Wars'
1: Vet Clinic Pays Tribute To Late Pup In Sweetest Way
2: On Gawker’s Problem With Women
3: HuffPost Rise: What You Need To Know On April 15
4: Sherry Xie, Student, And Her DIY No-Poo Shampoo
CPU times: user 287 ms, sys: 27 ms, total: 314 ms
Wall time: 394 ms
